In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = "/Users/AnaPSilva/Documents/Ana/Ironhack/Bootcamp/Final_Project/Data/Poem_Play/oscar_wilde.txt"

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 210525 characters


In [4]:
## removing page nr from the txt file (only for the Oscar Wilde txt)
text = text.replace('p. ','')

# Take a look at the first 250 characters in text
print(text[:250])

POEMS
3HÉLAS!
To drift with every passion till my soul
Is a stringed lute on which all winds can play,
Is it for this that I have given away
Mine ancient wisdom, and austere control?
Methinks my life is a twice-written scroll
Scrawled over on some bo


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

132 unique characters


In [6]:
## Before training, you need to convert the strings to a numerical representation.
## convert each character into a numeric ID. 
## It just needs the text to be split into tokens first.
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-03-15 00:01:25.441723: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
## from tokens to character IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[48, 49, 50, 51, 52, 53, 54], [71, 72, 73]]>

In [8]:
## it will also be important to invert this representation and 
## recover human-readable strings from it.
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
## join the characters back into strings.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  """join the characters back into strings"""
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

#### Create training examples and targets

In [12]:
## convert the text vector into a stream of character indices.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(209646,), dtype=int64, numpy=array([37, 36, 26, ..., 59, 59,  8])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

P
O
E
M
S


3
H
É
L


In [14]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

2075

In [15]:
## The batch method lets you easily convert 
## these individual characters to sequences of the desired size.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'P' b'O' b'E' b'M' b'S' b'\n' b'3' b'H' b'\xc3\x89' b'L' b'A' b'S' b'!'
 b'\n' b'T' b'o' b' ' b'd' b'r' b'i' b'f' b't' b' ' b'w' b'i' b't' b'h'
 b' ' b'e' b'v' b'e' b'r' b'y' b' ' b'p' b'a' b's' b's' b'i' b'o' b'n'
 b' ' b't' b'i' b'l' b'l' b' ' b'm' b'y' b' ' b's' b'o' b'u' b'l' b'\n'
 b'I' b's' b' ' b'a' b' ' b's' b't' b'r' b'i' b'n' b'g' b'e' b'd' b' '
 b'l' b'u' b't' b'e' b' ' b'o' b'n' b' ' b'w' b'h' b'i' b'c' b'h' b' '
 b'a' b'l' b'l' b' ' b'w' b'i' b'n' b'd' b's' b' ' b'c' b'a' b'n' b' '
 b'p' b'l' b'a' b'y'], shape=(101,), dtype=string)


In [16]:
## It's easier to see what this is doing if you join the 
## tokens back into strings

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'POEMS\n3H\xc3\x89LAS!\nTo drift with every passion till my soul\nIs a stringed lute on which all winds can play'
b',\nIs it for this that I have given away\nMine ancient wisdom, and austere control?\nMethinks my life is'
b' a twice-written scroll\nScrawled over on some boyish holiday\nWith idle songs for pipe and virelay,\nWh'
b'ich do but mar the secret of the whole.\nSurely there was a time I might have trod\nThe sunlit heights,'
b' and from life\xe2\x80\x99s dissonance\nStruck one clear chord to reach the ears of God:\nIs that time dead? lo! w'


In [17]:
def split_input_target(sequence):
    """takes a sequence as input, duplicates, 
    and shifts it to align the input and label for each timestep"""
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [18]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'POEMS\n3H\xc3\x89LAS!\nTo drift with every passion till my soul\nIs a stringed lute on which all winds can pla'
Target: b'OEMS\n3H\xc3\x89LAS!\nTo drift with every passion till my soul\nIs a stringed lute on which all winds can play'


#### Create training batches

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

#### Build The Model

- **tf.keras.layers.Embedding:** The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- **tf.keras.layers.GRU:** A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- **tf.keras.layers.Dense:** The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
display(vocab_size)
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

132

In [21]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [22]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#### Try the model

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 133) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  34048     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  136325    
                                                                 
Total params: 4,108,677
Trainable params: 4,108,677
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

## This gives us, at each timestep, a prediction of the next character index
sampled_indices

array([ 58,  15,   2,  75,   6,  55, 119,  20, 118, 105, 110, 108,  33,
        92, 118,  29,  44,   2,  71, 125,  76,  90,  93,  24,  53,  37,
       101,  47,  20,  42,  61,  99, 101,  61,  75,  43, 129,  82,  82,
        78,  48,  27, 116, 112, 114,  66,  82,  34,   3,  29,  32,  62,
        76,  50,  71,  31, 132,  17,  37,  55,  31,  73,  72,   6, 129,
         1,   6,  51,  35,  75, 113,  10, 114,  62,  40,  72,  37, 109,
        17, 101,  18,  31, 109,  95, 103,  95,  43,  48,  29,  14, 132,
        54,   8, 102,  92,  28,  17, 119,  76,  47])

In [26]:
## Decode these to see the text predicted by this untrained model
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'hat hoarse cave of strife\nWhere my white soul first kissed the mouth of sin.\n\n185HUMANITAD\n187It is '

Next Char Predictions:
 b'k6 \xc3\x86,h\xcf\x89;\xcf\x87\xce\xb8\xce\xbd\xce\xbbL\xce\xa1\xcf\x87HW x\xe1\xbd\x96\xc3\x88\xce\x9f\xce\xa3CfP\xce\xb1Z;Un\xce\xae\xce\xb1n\xc3\x86V\xe1\xbd\xb8\xc3\xba\xc3\xba\xc3\xa6aF\xcf\x84\xcf\x80\xcf\x82s\xc3\xbaM!HKo\xc3\x88cxJ\xe2\x80\x998PhJzy,\xe1\xbd\xb8\n,dN\xc3\x86\xcf\x811\xcf\x82oSyP\xce\xbc8\xce\xb19J\xce\xbc\xce\xa7\xce\xb5\xce\xa7VaH5\xe2\x80\x99g.\xce\xb4\xce\xa1G8\xcf\x89\xc3\x88Z'


#### Train the model
- At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 133)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.8904133, shape=(), dtype=float32)


In [29]:
tf.exp(example_batch_mean_loss).numpy()

133.00853

In [30]:
## training procedure using the tf.keras.Model.compile method. 
## Use tf.keras.optimizers.Adam with default arguments and the loss function.
model.compile(optimizer='adam', loss=loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_poem_oscarwilde_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
32/32 [==============================] - 126s 4s/step - loss: 3.7498
Epoch 2/100
32/32 [==============================] - 116s 4s/step - loss: 2.8713
Epoch 3/100
32/32 [==============================] - 114s 4s/step - loss: 2.5231
Epoch 4/100
32/32 [==============================] - 111s 3s/step - loss: 2.3363
Epoch 5/100
32/32 [==============================] - 115s 4s/step - loss: 2.2299
Epoch 6/100
32/32 [==============================] - 115s 4s/step - loss: 2.1531
Epoch 7/100
32/32 [==============================] - 115s 4s/step - loss: 2.0774
Epoch 8/100
32/32 [==============================] - 117s 4s/step - loss: 2.0061
Epoch 9/100
32/32 [==============================] - 104s 3s/step - loss: 1.9417
Epoch 10/100
32/32 [==============================] - 118s 4s/step - loss: 1.8824
Epoch 11/100
32/32 [==============================] - 118s 4s/step - loss: 1.8287
Epoch 12/100
32/32 [==============================] - 121s 4s/step - loss: 1.7776
Epoch 13/100
32/32 [=====

In [33]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [34]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [35]:
start = time.time()
states = None
next_char = tf.constant(['TIME '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

TIME FAhS A Deman more for your halm:
The gaudy leonine sunflower
   Hangs black and barren more a girl bask down teer becine
   And made the gaudy web of noon is spun
   By its twelve maidens, through the crimson haze
Frends dif the poet’s leaves tree upon a grass
   And loftiest culture I would stand apart,
Neither for God, nor red and brown;

122And he is rich, and fat and fleecy herds
   Of bleed and blackbird finds a mate
   And overstay the swallow twittering on the eaves
   At daybreak, when the mountain’s scarpèd feet upon his ears:
With blood of goats and blood of steers are ay
   But till the houngrage of day
   By his idon dances about the wattled fold.

And when they reached the strait Symplegades
   They beached their galley on the shore,
And where their hornèd master sits in state
Bring straig of all the grassy field,
For many a lovely coronal our northern isle finders with their jingling keys
   Opened each listening cardless fire unine soul,
And take a tuge perchance it

In [36]:
tf.saved_model.save(one_step_model, 'Oscar Wilde')

2022-03-15 03:14:21.924001: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: Oscar Wilde/assets


INFO:tensorflow:Assets written to: Oscar Wilde/assets
